In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from functools import reduce

### Import datasets

In [2]:
strokeDeathRate = pd.read_csv("data/stroke-death-rates-by-age.csv")[["Entity", "Year", "Deaths - Stroke - Sex: Both - Age: All Ages (Rate)"]]
strokeDeathRate.columns = ["country", "year", "stroke rate (per 100,000)"]
strokeDeathRate

,country,year,"stroke rate (per 100,000)"
0,Afghanistan,1990,105.721745
1,Afghanistan,1991,97.544198
2,Afghanistan,1992,86.763020
3,Afghanistan,1993,81.625506
4,Afghanistan,1994,80.469175
...,...,...,...
6835,Zimbabwe,2015,35.760842
6836,Zimbabwe,2016,35.561224
6837,Zimbabwe,2017,35.322171
6838,Zimbabwe,2018,35.133755


In [3]:
def meltYears(df):
    return df.melt(id_vars='country', var_name='year').sort_values(['country', 'year'])

In [4]:
def GdpToNum(gdp):
    if pd.isnull(gdp) or gdp[0].isalpha():
        return gdp
    if gdp[-1].isdigit():
        return float(gdp)
    if gdp[-1] == 'k':
        return float(gdp[:-1]) * 1000
    if gdp[-1] == 'm':
        return float(gdp[:-1]) * 1000000
gdpPerCapita = meltYears(pd.read_csv("data/gdppercapita_us_inflation_adjusted.csv").applymap(GdpToNum)).rename(columns={'value': 'GDP per capita'})
gdpPerCapita['year'] = gdpPerCapita['year'].apply(lambda x: int(x))
countries = gdpPerCapita['country'].unique()
gdpPerCapita

,country,year,GDP per capita
1,Afghanistan,1959,NaN
211,Afghanistan,1960,NaN
421,Afghanistan,1961,NaN
631,Afghanistan,1962,NaN
841,Afghanistan,1963,NaN
...,...,...,...
11969,Zimbabwe,2015,1430.0
12179,Zimbabwe,2016,1480.0
12389,Zimbabwe,2017,1530.0
12599,Zimbabwe,2018,1410.0


In [5]:
healthSpending = meltYears(pd.read_csv("data/government_health_spending_per_person_international_dollar.csv")).rename(columns={'value': 'health spending per person'})
healthSpending['year'] = healthSpending['year'].apply(lambda x: int(x))
healthSpending

,country,year,health spending per person
0,Afghanistan,1994,NaN
192,Afghanistan,1995,NaN
384,Afghanistan,1996,NaN
576,Afghanistan,1997,NaN
768,Afghanistan,1998,NaN
...,...,...,...
2303,Zimbabwe,2005,NaN
2495,Zimbabwe,2006,NaN
2687,Zimbabwe,2007,NaN
2879,Zimbabwe,2008,NaN


In [6]:
sexRatio = meltYears(pd.read_csv("data/sex_ratio_all_age_groups.csv")).rename(columns={'value': 'sex ratio'})
menBmi = meltYears(pd.read_csv("data/body_mass_index_bmi_men_kgperm2.csv")).rename(columns={'value': 'men bmi'})
womenBmi = meltYears(pd.read_csv("data/body_mass_index_bmi_women_kgperm2.csv")).rename(columns={'value': 'women bmi'})
bmiInfoDataset = [sexRatio, menBmi, womenBmi]
bmiAll = reduce(lambda left,right: pd.merge(left,right,on=['country', 'year'], how='inner'), bmiInfoDataset)
bmiAll['year'] = bmiAll['year'].apply(lambda x: int(x))
bmiAll

,country,year,sex ratio,men bmi,women bmi
0,Afghanistan,1979,103.0,21.5,20.4
1,Afghanistan,1980,NaN,21.5,20.5
2,Afghanistan,1981,NaN,21.4,20.5
3,Afghanistan,1982,NaN,21.4,20.6
4,Afghanistan,1983,NaN,21.4,20.6
...,...,...,...,...,...
5447,Zimbabwe,2003,NaN,21.9,24.6
5448,Zimbabwe,2004,91.9,21.9,24.6
5449,Zimbabwe,2005,NaN,21.9,24.6
5450,Zimbabwe,2006,NaN,22.0,24.6


In [7]:
def BmiBothSex(row):
    sexRatio = row[2]
    menBmi = row[3]
    womenBmi = row[4]
    if pd.isnull(sexRatio):
        return (menBmi + womenBmi) / 2
    else:
        return (menBmi * sexRatio + womenBmi * 100) / (sexRatio + 100)
bmiAll['average bmi'] = bmiAll.apply(BmiBothSex, axis=1)
averageBmi = bmiAll[['country', 'year', 'average bmi']]
averageBmi

,country,year,average bmi
0,Afghanistan,1979,20.958128
1,Afghanistan,1980,21.000000
2,Afghanistan,1981,20.950000
3,Afghanistan,1982,21.000000
4,Afghanistan,1983,21.000000
...,...,...,...
5447,Zimbabwe,2003,23.250000
5448,Zimbabwe,2004,23.306983
5449,Zimbabwe,2005,23.250000
5450,Zimbabwe,2006,23.300000


In [8]:
agedPopulation = meltYears(pd.read_csv("data/population_aged_60plus_years_both_sexes_percent.csv")).rename(columns={'value': 'percentage of population 60+'})
agedPopulation['year'] = agedPopulation['year'].apply(lambda x: int(x))
agedPopulation

,country,year,percentage of population 60+
1,Afghanistan,1949,5.03
202,Afghanistan,1950,5.05
403,Afghanistan,1951,5.05
604,Afghanistan,1952,5.03
805,Afghanistan,1953,4.99
...,...,...,...
29546,Zimbabwe,2095,21.40
29747,Zimbabwe,2096,21.70
29948,Zimbabwe,2097,22.00
30149,Zimbabwe,2098,22.30


In [9]:
# Not included
workingHours = meltYears(pd.read_csv("data/working_hours_per_week.csv")).rename(columns={'value': 'working hours per week'})
workingHours['year'] = workingHours['year'].apply(lambda x: int(x))
workingHours

,country,year,working hours per week
1,Afghanistan,1968,NaN
167,Afghanistan,1969,NaN
333,Afghanistan,1970,NaN
499,Afghanistan,1971,NaN
665,Afghanistan,1972,NaN
...,...,...,...
7967,Zimbabwe,2015,NaN
8133,Zimbabwe,2016,NaN
8299,Zimbabwe,2017,NaN
8465,Zimbabwe,2018,36.6


In [10]:
# Not included
airPolution = meltYears(pd.read_csv("data/en_atm_pm25_mc_m3.csv")).rename(columns={'value': 'mean annual exposure to PM2.5'})
airPolution['year'] = airPolution['year'].apply(lambda x: int(x))
airPolution

,country,year,mean annual exposure to PM2.5
0,Afghanistan,1989,65.5
194,Afghanistan,1990,NaN
388,Afghanistan,1991,NaN
582,Afghanistan,1992,NaN
776,Afghanistan,1993,NaN
...,...,...,...
4655,Zimbabwe,2012,23.1
4849,Zimbabwe,2013,22.8
5043,Zimbabwe,2014,21.7
5237,Zimbabwe,2015,21.7


In [30]:
dataFrames = [gdpPerCapita, healthSpending, averageBmi, agedPopulation, strokeDeathRate]
finalDataset = reduce(lambda left,right: pd.merge(left,right,on=['country', 'year'], how='outer'), dataFrames)
finalDataset = finalDataset.loc[(finalDataset['year'] >= 1994) & (finalDataset['year'] <= 2007) & (finalDataset['country'].isin(countries))]
finalDataset

,country,year,GDP per capita,health spending per person,average bmi,percentage of population 60+,"stroke rate (per 100,000)"
35,Afghanistan,1994,NaN,NaN,20.755769,3.80,80.469175
36,Afghanistan,1995,NaN,NaN,20.750000,3.79,78.684570
37,Afghanistan,1996,NaN,NaN,20.700000,3.77,77.295853
38,Afghanistan,1997,NaN,NaN,20.700000,3.75,76.340677
39,Afghanistan,1998,NaN,NaN,20.700000,3.73,76.450775
...,...,...,...,...,...,...,...
12793,Zimbabwe,2003,1210.0,NaN,23.250000,4.74,34.487922
12794,Zimbabwe,2004,1140.0,NaN,23.306983,4.72,35.655800
12795,Zimbabwe,2005,1090.0,NaN,23.250000,4.68,35.846474
12796,Zimbabwe,2006,1040.0,NaN,23.300000,4.62,36.540480


In [33]:
countryWithNoMissingData = []
for country in countries:
    if finalDataset.loc[finalDataset['country'] == country].isna().sum().sum() == 0:
        countryWithNoMissingData.append(country)
finalDataset = finalDataset.loc[finalDataset['country'].isin(countryWithNoMissingData)]
finalDataset

,country,year,GDP per capita,health spending per person,average bmi,percentage of population 60+,"stroke rate (per 100,000)"
96,Albania,1994,1490.0,52.9,25.097057,9.53,84.264694
97,Albania,1995,1630.0,57.8,25.150000,9.73,89.654892
98,Albania,1996,1460.0,61.2,25.150000,9.93,95.222384
99,Albania,1997,1600.0,72.7,25.200000,10.10,96.482943
100,Albania,1998,1820.0,97.5,25.300000,10.30,97.154950
...,...,...,...,...,...,...,...
12732,Zambia,2003,879.0,41.3,21.500000,3.56,43.982095
12733,Zambia,2004,918.0,44.5,21.562791,3.50,43.805623
12734,Zambia,2005,965.0,47.6,21.700000,3.46,43.454108
12735,Zambia,2006,1020.0,42.9,21.750000,3.42,43.509320


In [32]:
finalDataset.to_csv("data/final_dataset.csv")